In [1]:
print("Hello")

Hello


In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from pinecone import Pinecone as pin
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter  #to create chunks
from langchain.prompts import PromptTemplate
from langchain.llms import ctransformers


In [5]:
import os
os.environ['PINECONE_API_KEY']="18e96d9e-fdcc-4126-9e9d-20f386ebeb2a"
os.environ['PINECONE_API_ENV']="gcp-starter"

In [6]:
def load_pdf(data): 
    loader=DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    document=loader.load()
    return document    

In [7]:
%pwd

'c:\\Users\\adminn\\Documents\\GitHub\\Medical-chatbot-with-LLAM2'

In [8]:
extracted_data=load_pdf("data")

In [9]:
# transforming documents to chunks

def text_split(extracted_data):

    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=200)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks


In [10]:
text_chunks=text_split(extracted_data=extracted_data)
print(f"length of chunks {len(text_chunks)}")

length of chunks 1753


In [11]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embedding=download_hugging_face_embeddings()

In [13]:
embedding


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [14]:
query=embedding.embed_query("Hello world")
print(len(query)) #dimension of vector representation

384


In [15]:
from langchain_pinecone import PineconeVectorStore


In [16]:
# pinecone=pin(api_key=PINECONE_API_KEY,
#               environment=PINECONE_API_ENV)
from langchain_pinecone import PineconeVectorStore

index_name="medical-chatbot"
docsearch = PineconeVectorStore.from_documents(
    text_chunks,
    embedding,
    index_name = index_name
)

In [17]:
query = "c'est quoi la méoise?"

results=docsearch.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)


In [18]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [19]:
PROMPT=PromptTemplate(template=prompt_template,input_variables=["context","question"])
chain_type_kwargs={"prompt":PROMPT}

In [20]:
from langchain.llms import CTransformers
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={"max_new_tokens":512,
                          "temperature":0.8})

In [21]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract method _get_relevant_documents (type=type_error)